In [119]:
import json
from os.path import join
import os

import pandas as pd

from cogspaces.pipeline import get_output_dir

from json import JSONDecodeError

basedir_ids = [21]
basedirs = [join(get_output_dir(), 'predict_multi', str(_id), 'run') for _id in basedir_ids]
res_list = []
for basedir in basedirs:
    for exp_dir in os.listdir(basedir):
        exp_dir = join(basedir, exp_dir)
        try:
            config = json.load(open(join(exp_dir, 'config.json'), 'r'))
            info = json.load(open(join(exp_dir, 'info.json'), 'r'))
        except (JSONDecodeError, FileNotFoundError):
            continue
        datasets = config['datasets']
        datasets = '__'.join(datasets)
        alpha = config['alpha']
        beta = config['beta']
        seed = config['seed']
        model = config['model']
        source = config['source']
        score = info['score']
        rank = info['rank']
        normalized = config['with_std']
        res = {'datasets': datasets, 'alpha': alpha, 'beta': beta, 'seed': seed, 'rank': rank,
               'source': source, 'normalized': normalized,
              'model': model}
        for key, value in score.items():
            res[key] = value
        if basedir == join(get_output_dir(), 'predict_multi', '11', 'run') and model == 'logistic':
            continue
        res_list.append(res)
res = pd.DataFrame(res_list)

In [120]:
df_agg = res.groupby(by=['normalized', 'source', 'model', 'datasets', 'alpha', 'beta']).aggregate(['mean', 'std'])

In [121]:
df_agg

rank  \
                                                                         mean   
normalized source        model    datasets   alpha    beta                      
True       hcp_rs_concat logistic archi      0.000000 1.000000e-10  30.000000   
                                                      1.000000e-09  30.000000   
                                                      1.000000e-08  30.000000   
                                                      1.000000e-07  30.000000   
                                                      1.000000e-06  30.000000   
                                                      1.000000e-05  30.000000   
                                                      1.000000e-04  30.000000   
                                                      1.000000e-03  29.000000   
                                                      1.000000e-02  29.000000   
                                                      1.000000e-01  29.000000   
                         trace    archi      0.000000 0.000000e+00  30.000000   
                                             0.000001 0.000000e+00  30.000000   
                                             0.000003 0.000000e+00  30.000000   
                                             0.000010 0.000000e+00  30.000000   
                                             0.000032 0.000000e+00  30.000000   
                                             0.000100 0.000000e+00  30.000000   
                                             0.000316 0.000000e+00  30.000000   
                                             0.001000 0.000000e+00  25.333333   
                                             0.003162 0.000000e+00  24.000000   
                                             0.010000 0.000000e+00  22.333333   
                                             0.031623 0.000000e+00  21.666667   
                                             0.100000 0.000000e+00  12.000000   
                                             0.316228 0.000000e+00   5.000000   
                                             1.000000 0.000000e+00   1.000000   
                                  archi__hcp 0.000000 0.000000e+00  52.000000   
                                             0.000010 0.000000e+00  52.000000   
                                             0.000028 0.000000e+00  52.000000   
                                             0.000077 0.000000e+00  52.000000   
                                             0.000215 0.000000e+00  52.000000   
                                             0.000599 0.000000e+00  43.333333   
                                             0.001668 0.000000e+00  38.000000   
                                             0.004642 0.000000e+00  34.333333   
                                             0.012915 0.000000e+00  27.333333   
                                             0.035938 0.000000e+00  26.000000   
                                             0.100000 0.000000e+00  11.000000   

                                                                             \
                                                                        std   
normalized source        model    datasets   alpha    beta                    
True       hcp_rs_concat logistic archi      0.000000 1.000000e-10  0.00000   
                                                      1.000000e-09  0.00000   
                                                      1.000000e-08  0.00000   
                                                      1.000000e-07  0.00000   
                                                      1.000000e-06  0.00000   
                                                      1.000000e-05  0.00000   
                                                      1.000000e-04  0.00000   
                                                      1.000000e-03  0.00000   
                                                      1.000000e-02  0.00000   
                                              

In [122]:
indices = df_agg[('test_archi', 'mean')].groupby(level=['normalized', 'source', 'model', 'datasets']).aggregate('idxmax')

In [123]:
indices.values

array([(True, 'hcp_rs_concat', 'logistic', 'archi', 0.0, 0.01),
       (True, 'hcp_rs_concat', 'trace', 'archi', 0.01, 0.0),
       (True, 'hcp_rs_concat', 'trace', 'archi__hcp', 0.012915496650148827, 0.0)], dtype=object)

In [124]:
df_agg.loc[indices.values]

rank  \
                                                                 mean   
normalized source        model    datasets   alpha    beta              
True       hcp_rs_concat logistic archi      0.000000 0.01  29.000000   
                         trace    archi      0.010000 0.00  22.333333   
                                  archi__hcp 0.012915 0.00  27.333333   

                                                                     \
                                                                std   
normalized source        model    datasets   alpha    beta            
True       hcp_rs_concat logistic archi      0.000000 0.01  0.00000   
                         trace    archi      0.010000 0.00  0.57735   
                                  archi__hcp 0.012915 0.00  0.57735   

                                                                  seed  \
                                                                  mean   
normalized source        model    datasets   alpha    beta               
True       hcp_rs_concat logistic archi      0.000000 0.01  1243615929   
                         trace    archi      0.010000 0.00  1243615929   
                                  archi__hcp 0.012915 0.00  1243615929   

                                                                          \
                                                                     std   
normalized source        model    datasets   alpha    beta                 
True       hcp_rs_concat logistic archi      0.000000 0.01  6.141929e+08   
                         trace    archi      0.010000 0.00  6.141929e+08   
                                  archi__hcp 0.012915 0.00  6.141929e+08   

                                                           test_archi  \
                                                                 mean   
normalized source        model    datasets   alpha    beta              
True       hcp_rs_concat logistic archi      0.000000 0.01   0.837892   
                         trace    archi      0.010000 0.00   0.842735   
                                  archi__hcp 0.012915 0.00   0.854131   

                                                                      \
                                                                 std   
normalized source        model    datasets   alpha    beta             
True       hcp_rs_concat logistic archi      0.000000 0.01  0.006472   
                         trace    archi      0.010000 0.00  0.000855   
                                  archi__hcp 0.012915 0.00  0.013577   

                                                            test_hcp  \
                                                                mean   
normalized source        model    datasets   alpha    beta             
True       hcp_rs_concat logistic archi      0.000000 0.01       NaN   
                         trace    archi      0.010000 0.00       NaN   
                                  archi__hcp 0.012915 0.00  0.885342   

                                                                      \
                                                                 std   
normalized source        model    datasets   alpha    beta             
True       hcp_rs_concat logistic archi      0.000000 0.01       NaN   
                         trace    archi      0.010000 0.00       NaN   
                                  archi__hcp 0.012915 0.00  0.006259   

                                                           train_archi  \
                                                                  mean   
normalized source        model    datasets   alpha    beta               
True       hcp_rs_concat logistic archi      0.000000 0.01    0.997721   
                         trace    archi      0.010000 0.00    0.998291   
                                  archi__hcp 0.012915 0.00    0.973789   

                                                                      \
                                          

In [125]:
idx = pd.IndexSlice
single = res.set_index(['normalized', 'source', 'model', 'datasets', 'alpha', 'beta', 'seed'])
single.sort_index(inplace=True)
single_indices = [(*index, slice(None, None, None)) for index in indices]
pd.concat([single.loc[single_index] for single_index in single_indices], axis=0,
         keys=indices.values, names=single.index.names[:6])

rank  \
normalized source        model    datasets   alpha    beta seed               
True       hcp_rs_concat logistic archi      0.000000 0.01 636201818     29   
                                                           1230275727    29   
                                                           1864370243    29   
                         trace    archi      0.010000 0.00 636201818     22   
                                                           1230275727    22   
                                                           1864370243    23   
                                  archi__hcp 0.012915 0.00 636201818     27   
                                                           1230275727    27   
                                                           1864370243    28   

                                                                       test_archi  \
normalized source        model    datasets   alpha    beta seed                     
True       hcp_rs_concat logistic archi      0.000000 0.01 636201818     0.835043   
                                                           1230275727    0.845299   
                                                           1864370243    0.833333   
                         trace    archi      0.010000 0.00 636201818     0.842735   
                                                           1230275727    0.841880   
                                                           1864370243    0.843590   
                                  archi__hcp 0.012915 0.00 636201818     0.861538   
                                                           1230275727    0.838462   
                                                           1864370243    0.862393   

                                                                       test_hcp  \
normalized source        model    datasets   alpha    beta seed                   
True       hcp_rs_concat logistic archi      0.000000 0.01 636201818        NaN   
                                                           1230275727       NaN   
                                                           1864370243       NaN   
                         trace    archi      0.010000 0.00 636201818        NaN   
                                                           1230275727       NaN   
                                                           1864370243       NaN   
                                  archi__hcp 0.012915 0.00 636201818   0.890479   
                                                           1230275727  0.887177   
                                                           1864370243  0.878371   

                                                                       train_archi  \
normalized source        model    datasets   alpha    beta seed                      
True       hcp_rs_concat logistic archi      0.000000 0.01 636201818      0.998291   
                                                           1230275727     0.997436   
                                                           1864370243     0.997436   
                         trace    archi      0.010000 0.00 636201818      0.998291   
                                                           1230275727     0.998291   
                                                           1864370243     0.998291   
                                  archi__hcp 0.012915 0.00 636201818      0.972650   
                                                           1230275727     0.972650   
                                                           1864370243     0.976068   

                                                                       train_hcp  
normalized source        model    datasets   alpha    beta seed                   
True       hcp_rs_concat logistic archi      0.000000 0.01 636201818         NaN  
                                                           1230275727        NaN  
                                                           1864370243   

In [126]:
single.index.names

FrozenList(['normalized', 'source', 'model', 'datasets', 'alpha', 'beta', 'seed'])

In [116]:
import json
from os.path import join

import pandas as pd

from cogspaces.pipeline import get_output_dir

from json import JSONDecodeError

basedirs = [join(get_output_dir(), 'predict_trace_multi', '2', 'run')]
res_list = []
for basedir in basedirs:
    for i in range(1, 1000):
        exp_dir = join(basedir, str(i))
        try:
            config = json.load(open(join(exp_dir, 'config.json'), 'r'))
            info = json.load(open(join(exp_dir, 'info.json'), 'r'))
        except (JSONDecodeError, FileNotFoundError):
            continue
        datasets = config['datasets']
        datasets = '__'.join(datasets)
        alpha = config['alpha']
        max_iter = config['max_iter']
        beta = config['beta']
        seed = config['seed']
        with_std = config['with_std']
        score = info['score']
        source = config['source']
        rank = info['rank']
        res = {'datasets': datasets, 'source': source, 'alpha': alpha, 'beta': beta, 'seed': seed, 'rank': rank,
              'with_std': with_std}
        for key, value in score.items():
            res[key] = value
        res_list.append(res)
res = pd.DataFrame(res_list)

In [118]:
res.groupby(by=['source', 'with_std', 'alpha', 'beta', 'datasets']).aggregate(['mean', 'std'])

rank                  seed  \
                                                   mean       std        mean   
source          with_std alpha    beta datasets                                 
hcp_rs_positive False    0.00000  0    archi__hcp  52.0  0.000000  2303115213   
                         0.00001  0    archi__hcp  52.0  0.000000  2303115213   
                         0.00010  0    archi__hcp  52.0  0.000000  2303115213   
                         0.00100  0    archi__hcp  27.7  1.251666  2303115213   
                         0.01000  0    archi__hcp  13.5  0.849837  2303115213   
                         0.10000  0    archi__hcp   1.0  0.000000  2303115213   
                         1.00000  0    archi__hcp   0.0  0.000000  2303115213   
                         10.00000 0    archi__hcp   0.0  0.000000  2303115213   
                True     0.00000  0    archi__hcp  52.0  0.000000  2422319601   
                         0.00001  0    archi__hcp  52.0  0.000000  2303115213   
                         0.00010  0    archi__hcp  52.0  0.000000  2303115213   
                         0.00100  0    archi__hcp  50.0  1.247219  2303115213   
                         0.01000  0    archi__hcp  28.9  0.737865  2303115213   
                         0.10000  0    archi__hcp  11.7  0.483046  2303115213   
                         1.00000  0    archi__hcp   1.0  0.000000  2303115213   
                         10.00000 0    archi__hcp   0.0  0.000000  2303115213   

                                                                test_archi  \
                                                            std       mean   
source          with_std alpha    beta datasets                              
hcp_rs_positive False    0.00000  0    archi__hcp  1.156025e+09   0.841966   
                         0.00001  0    archi__hcp  1.156025e+09   0.842308   
                         0.00010  0    archi__hcp  1.156025e+09   0.845556   
                         0.00100  0    archi__hcp  1.156025e+09   0.851795   
                         0.01000  0    archi__hcp  1.156025e+09   0.700855   
                         0.10000  0    archi__hcp  1.156025e+09   0.067009   
                         1.00000  0    archi__hcp  1.156025e+09   0.033333   
                         10.00000 0    archi__hcp  1.156025e+09   0.033333   
                True     0.00000  0    archi__hcp  1.159131e+09   0.814340   
                         0.00001  0    archi__hcp  1.156025e+09   0.814615   
                         0.00010  0    archi__hcp  1.156025e+09   0.816496   
                         0.00100  0    archi__hcp  1.156025e+09   0.835897   
                         0.01000  0    archi__hcp  1.156025e+09   0.839829   
                         0.10000  0    archi__hcp  1.156025e+09   0.713761   
                         1.00000  0    archi__hcp  1.156025e+09   0.085556   
                         10.00000 0    archi__hcp  1.156025e+09   0.033333   

                                                             test_hcp  \
                                                        std      mean   
source          with_std alpha    beta datasets                         
hcp_rs_positive False    0.00000  0    archi__hcp  0.011176  0.924816   
                         0.00001  0    archi__hcp  0.011623  0.924981   
                         0.00010  0    archi__hcp  0.011217  0.924815   
                         0.00100  0    archi__hcp  0.017582  0.906463   
                         0.01000  0    archi__hcp  0.016956  0.737993   
                         0.10000  0    archi__hcp  0.001407  0.109892   
                         1.00000  0    archi__hcp  0.000000  0.043538   
                         10.00000 0    archi__hcp  0.000000  0.043483   
                True     0.00000  0    archi__hcp  0.012847  0.913146   
                         0.00001  0    archi__hcp  0.011860  0.913026   
                         0.00010  0    archi__hcp  0.012195  0.913412   
      

In [26]:
import json
from os.path import join

import pandas as pd

from cogspaces.pipeline import get_output_dir

from json import JSONDecodeError

basedirs = [join(get_output_dir(), 'predict_non_convex_multi', '1', 'run')]
res_list = []
for basedir in basedirs:
    for i in range(1, 1000):
        exp_dir = join(basedir, str(i))
        try:
            config = json.load(open(join(exp_dir, 'config.json'), 'r'))
            info = json.load(open(join(exp_dir, 'info.json'), 'r'))
        except (JSONDecodeError, FileNotFoundError):
            continue
        datasets = config['datasets']
        datasets = '__'.join(datasets)
        n_components = config['n_components']
        max_iter = config['max_iter']
        dropout = config['input_dropout_rate']
        seed = config['seed']
        with_std = config['with_std']
        score = info['score']
        source = config['source']
        rank = info['rank']
        res = {'datasets': datasets, 'source': source, 'n_components': n_components, 'dropout': dropout, 'seed': seed, 'rank': rank,
              'with_std': with_std}
        for key, value in score.items():
            res[key] = value
        res_list.append(res)
res = pd.DataFrame(res_list)

In [27]:
res

""


In [24]:
res.groupby(by=['n_components', 'dropout']).aggreagate('mean')

KeyError: 'n_components'